### Selenium 설치

In [1]:
!pip install selenium

### Selenium Standalone Server 다운로드
- 크롬 브라우저 버전 확인 [메뉴] - [도움말] - [Chrome 정보]
- 크롬 브라우저로 접속 https://www.selenium.dev/downloads/
- ipynb 파일 위치에 다운로드

### Webdriver 다운로드
- 브라우저 버전 확인 [메뉴] - [도움말] - [Chrome정보]
- https://sites.google.com/a/chromium.org/chromedriver/downloads 접속하여 크롬 브라우저 버전에 맞는 웹 드라이버 다운로드 및 현재 디렉토리 다운로드

### Webdriver 로드 및 브라우저 기동

In [43]:
from selenium.webdriver import Chrome
import time
import sqlite3
from pandas.io import sql
import os
import pandas as pd

In [44]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized"); # --start-maximized : 웹 브라우저의 크기를 최대화하는 명령어

browser = webdriver.Chrome('chromedriver', options=options)

In [45]:
browser.get('https://www.data.go.kr/')
browser.implicitly_wait(5) # implicitly_wait : 잠깐 대기(로딩이 되기위해서)

#### 로그인 하기

In [46]:
browser.find_element_by_xpath('//*[@id="header"]/div/div/div/div[2]/div/a[1]').click()
browser.implicitly_wait(5)

In [47]:
browser.find_element_by_xpath('//*[@id="mberId"]').send_keys('gyng12')  # ID

In [48]:
browser.find_element_by_xpath('//*[@id="pswrd"]').send_keys('!rkekak12345') # PW

In [49]:
browser.find_element_by_xpath('//*[@id="loginVo"]/div[2]/div[2]/div[2]/div/div[1]/button').click()
browser.implicitly_wait(5)

#### 팝업창 제거

In [50]:
browser.find_element_by_xpath('//*[@id="layer_popup_info_0"]/div[1]/a').click()
browser.find_element_by_xpath('//*[@id="layer_popup_info_1"]/div[1]/a').click()

#### 정보공유 자료실 링크 클릭

In [51]:
browser.find_element_by_xpath('//*[@id="M000400_pc"]/a').click()   

In [52]:
browser.find_element_by_xpath('//*[@id="M000402_pc"]/a').click()

#### 정보공유 자료실 데이터 추출

In [53]:
def db_save(ARTICLE_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            ARTICLE_LIST.to_sql(name = 'ARTICLE_LIST', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(ARTICLE_LIST), '건 저장완료..')

#### 자료실 데이터 추출 및 저장

In [54]:
trs = browser.find_elements_by_xpath('//*[@id="searchVO"]/div[5]/table/tbody/tr')
df_list = []
for tr in trs:
    df = pd.DataFrame({
            'NO': [tr.find_element_by_xpath('td[1]').text],
            'TITLE': [tr.find_element_by_xpath('td[2]').text],
            'IQRY': [tr.find_element_by_xpath('td[3]').text],
            'REGDT': [tr.find_element_by_xpath('td[4]').text],
            'CHGDT': [tr.find_element_by_xpath('td[5]').text],
        })
    df_list.append(df)
    
ARTICLE_LIST = pd.concat(df_list)
db_save(ARTICLE_LIST)

10 건 저장완료..


#### 자료실 글목록 상세보기 클릭

In [55]:
browser.find_element_by_xpath('//*[@id="searchVO"]/div[5]/table/tbody/tr[1]/td[2]/a').click()
browser.implicitly_wait(3)

#### 상세보기 첨부파일 다운로드 및 브라우저 종료

In [56]:
browser.find_element_by_xpath('//*[@id="recsroomDetail"]/div[2]/div[4]/div/a').click()
time.sleep(10)

In [57]:
browser.quit() # 브라우저를 종료해주어야 한다.